In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
import ftplib
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

ModuleNotFoundError: No module named 'arcpy'

In [4]:
sys.path

['',
 'C:\\Python27\\ArcGIS10.5\\python.exe',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\python36.zip',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3',
 'C:\\Users\\mr068144\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\mr068144\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\mr068144\\.ipython']

In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'C:\Users\mr068144\Downloads\FARS Browser'
ReceivedDir  = Project_DIR + '\\1.Downloaded'
Project_GDB  = Project_DIR + '\\2.Processed'

2018-07-20 12:59:20


In [3]:
def ConvertROAD_FUNtoFUNC_SYS(road_fun):
    if road_fun<10:
        rur_urb = 1
    elif road_fun>10 and road_fun<20:
        rur_urb = 2
    else:
        rur_urb = 9
    fuc_conv_dict = {1:[1,11],2:[12],3:[2,13],4:[3,14],5:[4,15],6:[5],7:[6,16],96:[],98:[],99:[9,19,99]}
    fun_sys = [i for i in fuc_conv_dict if road_fun in fuc_conv_dict[i]][0]
    return([rur_urb,fun_sys])

In [4]:
State_Dict = {
    1:"Alabama",
    2:"Alaska",
    4:"Arizona",
    5:"Arkansas",
    6:"California",
    8:"Colorado",
    9:"Connecticut",
    10:"Delaware",
    11:"District of Columbia",
    12:"Florida",
    13:"Georgia",
    15:"Hawaii",
    16:"Idaho",
    17:"Illinois",
    18:"Indiana",
    19:"Iowa",
    20:"Kansas",
    21:"Kentucky",
    22:"Louisiana",
    23:"Maine",
    24:"Maryland",
    25:"Massachusetts",
    26:"Michigan",
    27:"Minnesota",
    28:"Mississippi",
    29:"Missouri",
    30:"Montana",
    31:"Nebraska",
    32:"Nevada",
    33:"New Hampshire",
    34:"New Jersey",
    35:"New Mexico",
    36:"New York",
    37:"North Carolina",
    38:"North Dakota",
    39:"Ohio",
    40:"Oklahoma",
    41:"Oregon",
    42:"Pennsylvania",
    43:"Puerto Rico",
    44:"Rhode Island",
    45:"South Carolina",
    46:"South Dakota",
    47:"Tennessee",
    48:"Texas",
    49:"Utah",
    50:"Vermont",
    51:"Virginia",
    52:"Virgin Islands",
    53:"Washington",
    54:"West Virginia",
    55:"Wisconsin",
    56:"Wyoming",
}
ROAD_FUN_Dict = {
        1: 'Rural-Principal Arterial – Interstate',
        2: 'Rural-Principal Arterial – Other',
        3: 'Rural-Minor Arterial',
        4: 'Rural-Major Collector',
        5: 'Rural-Minor Collector',
        6: 'Rural-Local Road or Street',
        9: 'Rural-Unknown Rural',
        11: 'Urban-Principal Arterial – Interstate',
        12: 'Urban-Principal Arterial – Other (Freeways or Expressways)',
        13: 'Urban-Other Principal Arterial',
        14: 'Urban-Minor Arterial',
        15: 'Urban-Collector',
        16: 'Urban-Local Road or Street',
        19: 'Urban-Unknown Urban',
        99: 'Unknown',
}
FUN_SYS_Dict = {
        1: '01.Interstate',
        2: '02.Principal Arterial – Other Freeways and Expressways',
        3: '03.Principal Arterial – Other ',
        4: '04.Minor Arterial',
        5: '05.Major Collector',
        6: '06.Minor Collector',
        7: '07.Local',
        96: '08.Trafficway Not in State Inventory',
        98: '09.Not Reported',
        99: '10.Unknown'
}
RUR_URB_Dict = {
    1: '1.Rural',
    2: '2.Urban',
    6: '3.Trafficway Not in State Inventory',
    8: '4.Not Reported',
    9: '5.Unknown'
}

In [57]:
FARS_DF = pd.DataFrame()
for year in Years:
    df = pd.read_csv(ReceivedDir + '\\accident{}.csv'.format(year))
    df['YEAR'] = year
    if year<2015:
        df['FUNC_SYS'] = [ConvertROAD_FUNtoFUNC_SYS(i)[1] for i in list(df.ROAD_FNC)]
        df['RUR_URB' ] = [ConvertROAD_FUNtoFUNC_SYS(i)[0] for i in list(df.ROAD_FNC)]
    df['FC'] = [FUN_SYS_Dict[i] for i in list(df.FUNC_SYS)]
    df['RuralUrban'] = [RUR_URB_Dict[i] for i in list(df.RUR_URB)]
    FARS_DF = pd.concat([FARS_DF,df])

In [60]:
State_DF = FARS_DF[FARS_DF.STATE=={State_Dict[k]:k for k in State_Dict}['Illinois']]
Res = pd.DataFrame(State_DF.groupby(['YEAR','FC','RuralUrban'])['FATALS'].aggregate(np.sum))
Res = Res.sort_index(level=[0,1,2])
#Res.to_excel('FARS_FC_Urban.xlsx')
display(Res)

FATALS
YEAR FC                                                 RuralUrban                                 
2012 01.Interstate                                      1.Rural                                  51
                                                        2.Urban                                  59
     02.Principal Arterial – Other Freeways and Expr... 2.Urban                                  19
     03.Principal Arterial – Other                      1.Rural                                  62
                                                        2.Urban                                 174
     04.Minor Arterial                                  1.Rural                                  75
                                                        2.Urban                                 133
     05.Major Collector                                 1.Rural                                 115
                                                        2.Urban                                  83
     06.Minor Collector                                 1.Rural                                  10
     07.Local                                           1.Rural                                  82
                                                        2.Urban                                  91
     10.Unknown                                         2.Urban                                   2
2013 01.Interstate                                      1.Rural                                  61
                                                        2.Urban                                  65
     02.Principal Arterial – Other Freeways and Expr... 2.Urban                                   3
     03.Principal Arterial – Other                      1.Rural                                  61
                                                        2.Urban                                 210
     04.Minor Arterial                                  1.Rural                                 103
                                                        2.Urban                                 146
     05.Major Collector                                 1.Rural                                 106
                                                        2.Urban                                  92
     06.Minor Collector                                 1.Rural                                  10
     07.Local                                           1.Rural                                  70
                                                        2.Urban                                  64
2014 01.Interstate                                      1.Rural                                  50
                                                        2.Urban                                  67
     02.Principal Arterial – Other Freeways and Expr... 2.Urban                                   6
     03.Principal Arterial – Other                      1.Rural                                  83
                                                        2.Urban                                 185
...                                                                                             ...
     10.Unknown                                         2.Urban                                   4
                                                        5.Unknown                                 1
2015 01.Interstate                                      1.Rural                                  47
                                                        2.Urban                                  76
     02.Principal Arterial – Other Freeways and Expr... 1.Rural                                   3
                                                        2.Urban                                   8
     03.Principal Arterial – Other                      1.Rural                                  78
                                                        2.Urban                              

In [13]:
FARS_Occ_DF = pd.DataFrame()
for year in Years:
    df = pd.read_csv(ReceivedDir + '\\person{}.csv'.format(year))
    df['Year'] = year
    FARS_Occ_DF = pd.concat([FARS_Occ_DF,df])
FARS_Occ_DF['FARS_Ped_K'] = [{True:1,False:0}[v==4] for v in FARS_Occ_DF.INJ_SEV]
State_Occ_DF = FARS_Occ_DF[FARS_Occ_DF.STATE=={State_Dict[k]:k for k in State_Dict}['New Mexico']]
State_Ped_DF = State_Occ_DF[State_Occ_DF.PER_TYP==5]
pd.DataFrame(State_Ped_DF.groupby(['Year'])['FARS_Ped_K'].aggregate(np.sum))

In [20]:
FARS_Occ_DF['FARS_Ped_K'] = [{True:1,False:0}[v==4] for v in FARS_Occ_DF.INJ_SEV]
State_Occ_DF = FARS_Occ_DF[FARS_Occ_DF.STATE=={State_Dict[k]:k for k in State_Dict}['New Mexico']]
State_Ped_DF = State_Occ_DF[State_Occ_DF.PER_TYP==5]
pd.DataFrame(State_Ped_DF.groupby(['Year'])['FARS_Ped_K'].aggregate(np.sum))

,FARS_Ped_K
Year,
2012,75
2013,49
2014,75
2015,54
2016,73
